In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.linear_model import LinearRegression
from sklearn.metrics import classification_report

## IMPORTING FILE

In [ ]:
from google.colab import files
uploaded=files.upload()

Saving IMDb_Movies_India.csv to IMDb_Movies_India (9).csv


In [ ]:
data=pd.read_csv("IMDb_Movies_India.csv",encoding="ISO-8859-1")
data

,Name,Year,Duration,Genre,Rating,Votes,Director,Actor 1,Actor 2,Actor 3
0,,NaN,NaN,Drama,NaN,NaN,J.S. Randhawa,Manmauji,Birbal,Rajendra Bhatia
1,#Gadhvi (He thought he was Gandhi),(2019),109 min,Drama,7.0,8,Gaurav Bakshi,Rasika Dugal,Vivek Ghamande,Arvind Jangid
2,#Homecoming,(2021),90 min,"Drama, Musical",NaN,NaN,Soumyajit Majumdar,Sayani Gupta,Plabita Borthakur,Roy Angana
3,#Yaaram,(2019),110 min,"Comedy, Romance",4.4,35,Ovais Khan,Prateik,Ishita Raj,Siddhant Kapoor
4,...And Once Again,(2010),105 min,Drama,NaN,NaN,Amol Palekar,Rajat Kapoor,Rituparna Sengupta,Antara Mali
...,...,...,...,...,...,...,...,...,...,...
15504,Zulm Ko Jala Doonga,(1988),NaN,Action,4.6,11,Mahendra Shah,Naseeruddin Shah,Sumeet Saigal,Suparna Anand
15505,Zulmi,(1999),129 min,"Action, Drama",4.5,655,Kuku Kohli,Akshay Kumar,Twinkle Khanna,Aruna Irani
15506,Zulmi Raj,(2005),NaN,Action,NaN,NaN,Kiran Thej,Sangeeta Tiwari,NaN,NaN
15507,Zulmi Shikari,(1988),NaN,Action,NaN,NaN,NaN,NaN,NaN,NaN


## DATA CLEANING

In [ ]:
data.describe()

,Rating
count,7919.000000
mean,5.841621
std,1.381777
min,1.100000
25%,4.900000
50%,6.000000
75%,6.800000
max,10.000000


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15509 entries, 0 to 15508
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Name      15509 non-null  object 
 1   Year      14981 non-null  object 
 2   Duration  7240 non-null   object 
 3   Genre     13632 non-null  object 
 4   Rating    7919 non-null   float64
 5   Votes     7920 non-null   object 
 6   Director  14984 non-null  object 
 7   Actor 1   13892 non-null  object 
 8   Actor 2   13125 non-null  object 
 9   Actor 3   12365 non-null  object 
dtypes: float64(1), object(9)
memory usage: 1.2+ MB


In [ ]:
data.isnull().sum()

Name           0
Year         528
Duration    8269
Genre       1877
Rating      7590
Votes       7589
Director     525
Actor 1     1617
Actor 2     2384
Actor 3     3144
dtype: int64

In [ ]:
data["Votes"] = data["Votes"].replace("$5.16M", 516)
data["Votes"] = pd.to_numeric(data['Votes'].str.replace(',', ''))
data["Votes"].fillna(data["Votes"].mean(),inplace=True)
data['Votes'].isnull().sum()

0

In [ ]:
data['Duration']=data['Duration'].str.replace('min','').apply(pd.to_numeric)
data['Duration'].fillna(data['Duration'].mean(),inplace=True)
data['Duration'].isnull().sum()

0

In [ ]:
data.dropna(subset=['Director', 'Actor 1', 'Actor 2', 'Actor 3'], inplace=True)
data['Rating'].fillna(data['Rating'].mode().max(),inplace=True)
data.dropna(subset=['Year'],inplace=True)

# NOW ASSGIN RATING TO THE ACTORS DIRECTORS AND GENER BY CREARING THE NEW COLIUMN AND DELETE THE ORIGNAL COLUMNS

# # DROP ORIGINAL COLUMN

In [ ]:
numeric_columns = data.select_dtypes(include=['float64', 'int64']).columns
correlation_matrix = data[numeric_columns].corr(method='spearman')
print(data.columns)

Index(['Name', 'Year', 'Duration', 'Genre', 'Rating', 'Votes', 'Director',
       'Actor 1', 'Actor 2', 'Actor 3'],
      dtype='object')


In [ ]:
data.dropna(subset=['Director', 'Actor 1', 'Actor 2', 'Actor 3', 'Genre'], inplace=True)
(round(data.isnull().sum()/data.isnull().count(), 4)*100).sort_values(ascending=False)

Name        0.0
Year        0.0
Duration    0.0
Genre       0.0
Rating      0.0
Votes       0.0
Director    0.0
Actor 1     0.0
Actor 2     0.0
Actor 3     0.0
dtype: float64

In [ ]:
data['Year'] = data['Year'].str.extract('(\d+)').astype(float)

actor1=data.groupby('Actor 1').agg({'Rating' : 'mean'}).to_dict()['Rating']
actor2=data.groupby('Actor 2').agg({'Rating' : 'mean'}).to_dict()['Rating']
actor3=data.groupby('Actor 3').agg({'Rating' : 'mean'}).to_dict()['Rating']
director=data.groupby('Director').agg({'Rating' : 'mean'}).to_dict()['Rating']
genre=data.groupby('Genre').agg({'Rating' : 'mean'}).to_dict()['Rating']

data['actor1_col']=round(data['Actor 1'].map(actor1),1)
data['actor2_col']=round(data['Actor 2'].map(actor2),1)
data['actor3_col']=round(data['Actor 3'].map(actor3),1)
data['director_col']=round(data['Director'].map(director),1)
data['genre_col']=round(data['Genre'].map(genre),1)


In [ ]:
data.drop(['Actor 1','Actor 2','Actor 3','Director','Genre'],axis=1,inplace=True)

# TRAIN THE MODEL FIT TO IT AND TEST IT

In [ ]:
x=data[['Year','Duration','Votes','actor1_col','actor2_col','actor3_col','director_col','genre_col']]
y=data['Rating']

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)

In [ ]:
model=LinearRegression()

In [ ]:
data.isnull().sum()

Name            0
Year            0
Duration        0
Rating          0
Votes           0
actor1_col      0
actor2_col      0
actor3_col      0
director_col    0
genre_col       0
dtype: int64

In [ ]:
model.fit(x_train,y_train)

LinearRegression()

In [ ]:
y_pred=model.predict(x_test)

In [ ]:
from sklearn.metrics import mean_squared_error,mean_absolute_error,r2_score

In [ ]:
print("Mean square error : ",mean_squared_error(y_test,y_pred))
print()
print("mean absolute error : ",mean_absolute_error(y_test,y_pred))
print()
print("SCORE : ",r2_score(y_test,y_pred))

Mean square error :  0.3666298752786349

mean absolute error :  0.4299810396204221

SCORE :  0.7225057571173501
